In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, SimpleRNN, LSTM, TimeDistributed
import numpy as np

In [ ]:
# first data set
idx2char = ['h', 'i', 'e', 'l', 'o', '.']

h, i, e, l, o, dot = 0, 1, 2, 3, 4, 5

x = np.array([[[h, i, h], [i, h, e], [h, e, l], [e, l, l], [l, l, o]]], dtype=np.float32)
y = np.array([[[e], [l], [l], [o], [dot]]], dtype=np.float32)

print("x:", x, x.shape)
print("y:", y, y.shape)

x: [[[0. 1. 0.]
  [1. 0. 2.]
  [0. 2. 3.]
  [2. 3. 3.]
  [3. 3. 4.]]] (1, 5, 3)
y: [[[2.]
  [3.]
  [3.]
  [4.]
  [5.]]] (1, 5, 1)


In [ ]:
model_type = 1 # 1: Regression, 2: Classification

In [ ]:
x_input = Input(batch_shape=(None, x.shape[1], 3))
rnn = LSTM(6, return_sequences=True)(x_input)
print(rnn)
if model_type == 1:
  x_output = TimeDistributed(Dense(1))(rnn)
  #x_output = Dense(1)(rnn)
elif model_type == 2:
  x_output = TimeDistributed(Dense(4, activation="softmax"))(rnn)
  #x_output = Dense(4, activation="softmax")(rnn)

KerasTensor(type_spec=TensorSpec(shape=(None, 5, 6), dtype=tf.float32, name=None), name='lstm/PartitionedCall:1', description="created by layer 'lstm'")


In [ ]:
model = Model(x_input, x_output)

if model_type == 1:
  model.compile(loss='mean_squared_error', optimizer='adam')
elif model_type == 2:
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5, 3)]            0         
                                                                 
 lstm (LSTM)                 (None, 5, 6)              240       
                                                                 
 time_distributed (TimeDistr  (None, 5, 1)             7         
 ibuted)                                                         
                                                                 
Total params: 247
Trainable params: 247
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(x, y, epochs=1000, batch_size=1, verbose=0)

In [44]:
pred = model.predict(x, batch_size=1)
if model_type == 1:
  print(pred)
  pred = np.squeeze(pred, axis=0)
  # print(np.round(pred))
  idx = np.round(pred).astype(np.int8)
  idx = np.squeeze(idx)
  print("Argmax: ")
  print(idx)
  result = [idx2char[i] for i in idx]
  print("Result: ")
  print(result)
  # idx = np.round(pred).astype(np.int8)
  # print("Argmax: ")
  # print(idx)
  # result = [idx2char[i] for i in idx]
  # print("Result: ")
  # print(result)
elif model_type == 2:
  pred = np.squeeze(pred, axis=0)
  print(np.round(pred, 3))
  idx = [np.argmax(r) for r in pred]
  print(idx)

[[[1.4775937]
  [2.748584 ]
  [3.3513432]
  [4.267691 ]
  [4.6416607]]]
Argmax: 
[1 3 3 4 5]
Result: 
['i', 'l', 'l', 'o', '.']
